## Imports

In [ ]:
import os
import re
import sys
import typing
import gc
import pandas as pd
import pickle
import logging

sys.path.append(
    os.path.join('.','src')
)

from src.models import Pipeline
from src.nodes import *

In [ ]:
logging.basicConfig(filename='logs/logs.log', level=logging.DEBUG)

# Tweets Model

In [ ]:
pipeline = Pipeline('CONFIG_MODEL_TWEETS.json', load_model_data = True)

In [ ]:
d = {
    'train_set_len' : len(pipeline.train_dataset),
    'train_set_tokens' : pipeline.train_dataset.token_len(),
    'val_set_len' : len(pipeline.val_dataset),
    'val_set_tokens' : pipeline.val_dataset.token_len(),
    'test_set_len' : len(pipeline.test_dataset),
    'test_set_tokens' : pipeline.test_dataset.token_len(),
}

In [ ]:
d

In [ ]:
pipeline.perplexity()

In [ ]:
pipeline.train_model()

In [ ]:
pipeline.perplexity()

In [ ]:
pipeline = Pipeline('CONFIG_MODEL_TWEETS.json', load_model_data = False)

In [ ]:
pipeline.load_model('models/tweets/tweets.pth')

In [ ]:
pipeline.generate(start_text = 'all', num_words = 40)

In [ ]:
pipeline.generate(start_text = 'what', num_words = 40)

In [ ]:
pipeline.generate(start_text = 'i like', num_words = 40)

# Wiki Model

In [ ]:
pipeline = Pipeline('CONFIG_MODEL_WIKI.json', load_model_data = True)

In [ ]:
pipeline.perplexity(with_recall = True, with_tqdm = True)

In [ ]:
pipeline.load_model('models/wiki103/wiki103.pth')

In [ ]:
pipeline.perplexity(with_recall = True, with_tqdm = True)

In [ ]:
pipeline.train_model()

In [ ]:
pipeline.perplexity()

In [ ]:
pipeline.generate(start_text = 'all', num_words=40)

In [ ]:
pipeline.generate(start_text = 'what', num_words=40)

In [ ]:
pipeline.generate(start_text = 'i like')

In [ ]:
d = {
    'train_set_len' : len(pipeline.train_dataset),
    'train_set_tokens' : pipeline.train_dataset.token_len(),
    'val_set_len' : len(pipeline.val_dataset),
    'val_set_tokens' : pipeline.val_dataset.token_len(),
    'test_set_len' : len(pipeline.test_dataset),
    'test_set_tokens' : pipeline.test_dataset.token_len(),
}

In [ ]:
d

## FedAVG

In [1]:
from src.federated_pipeline import Federated_AVG

import os
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
federated = Federated_AVG(
    "CONFIG_MODEL_TWEETS.json",
    "CONFIG_FEDERATED_TWEETS.json",
    testing = True
)

100%|██████████| 50/50 [00:15<00:00,  3.14it/s]


In [3]:
a = np.array([len(n.data) for n in federated.nodes.values()])
a

array([2948, 2954, 2851, 4121, 4065, 3986, 1801, 4089, 1930, 3477, 2594,
       3342, 2995, 2657, 2675, 3628, 2156, 2323, 3309, 3575, 3177, 4176,
       2776, 2854, 3333, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000,
       3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000,
       3000, 3000, 3000, 3000, 3000, 3000])

In [4]:
federated.train(20, save_results = False)

  2%|▏         | 3/193 [00:00<00:06, 28.53it/s]

round 0


  0%|          | 0/50 [00:00<?, ?it/s]

round 1


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

round 19


  0%|          | 0/50 [00:00<?, ?it/s]

round 20


100%|██████████| 50/50 [00:00<00:00, 652.68it/s]


In [5]:
pd.DataFrame(federated.results)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
perplexity,82.280398,189.175464,312.735829,313.627502,330.508332,378.797200,395.273117,377.667653,377.596313,511.838256,...,540.186345,7488.576066,8820.806284,490.042779,540.147629,16000.969242,505.977410,807.295065,1656.615388,424.982982
loss,4.406816,5.239529,5.742709,5.745860,5.798581,5.936045,5.978216,5.931737,5.932259,6.235802,...,6.291490,8.916497,9.082331,6.193844,6.290457,9.676824,6.225039,6.691768,7.411625,6.050663
f1_recall,0.295508,0.225968,0.190668,0.190580,0.186166,0.173482,0.171231,0.176204,0.179279,0.156943,...,0.155825,0.059152,0.018525,0.128427,0.136299,0.039346,0.134592,0.132076,0.072836,0.147261
f3_recall,0.402290,0.332956,0.298657,0.293904,0.292815,0.277483,0.269066,0.277571,0.282412,0.253572,...,0.248422,0.111432,0.043628,0.214138,0.222539,0.086785,0.220509,0.216624,0.150027,0.238313
attack_perplexity,598.074532,444.141736,1208.317450,863.370627,574.217410,487.995960,970.634404,667.914356,572.748604,936.209325,...,1166.273667,1.000026,1.000000,9.211939,59.540461,1.000003,1.714386,6.365241,1.000007,1.291710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
attack_perplexity_25,598.074532,444.141736,1208.317450,863.370627,574.217410,487.995960,970.634404,667.914356,572.748604,936.209325,...,1166.273667,1.000026,1.000000,9.211939,59.540461,1.000003,1.714386,6.365241,1.000007,1.291710
train_len_25,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,...,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
train_tokens_25,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,...,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
len_25,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,...,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000


In [6]:
federated.general_model.generate(federated.vocabulary, 'all')

'all have to go stories on his trumps deserves the pin is not having an silent at the one of the other dc as this thread is spread there is the role and which can he had many of ppl to rump that the fact pm leaking health room is one of trumps anything he was very living in the rally this is short of his things that the leaving divide where it almost not the debate in major weeks million the money and foundation had been endorsed in their camera campaign greater source jim tape in unless the nyt form'

In [ ]:
plt.figure()
for k,v in pd.DataFrame(res[0]).T.to_dict().items():
    if 'perplexity_' in k:
        plt.plot(list(v.values()))

In [ ]:
plt.figure()
for k,v in pd.DataFrame(res[0]).T.to_dict().items():
    if 'f1_recall_' in k:
        plt.plot(list(v.values()))

In [ ]:
plt.figure()
for k,v in pd.DataFrame(res[0]).T.to_dict().items():
    if 'f3_recall_' in k:
        plt.plot(list(v.values()))

In [ ]:
from src.federated_pipeline import Federated_LICCHAVI

import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
federated = Federated_LICCHAVI(
    "CONFIG_MODEL_TWEETS.json",
    "CONFIG_FEDERATED_TWEETS.json",
    testing = True
)

In [ ]:
federated.train(5)

In [ ]:
dir_ = os.path.join('tweets', 'LICCHAVI')
res = {}
for i,name in enumerate(os.listdir(dir_)):
    with open(os.path.join(dir_, name, 'metrics.pickle'), 'rb') as f:
        res[i] = pickle.load(f)

In [ ]:
f, ax = plt.subplots(1)
ax.set_ylim([0, 300])
for k,v in pd.DataFrame(res[0]).T.to_dict().items():
    if 'perplexity' == k:
        plt.plot(list(v.values()))
        

In [ ]:
from src.federated_pipeline import Federated_LICCHAVI_avg

import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
federated = Federated_LICCHAVI_avg(
    "CONFIG_MODEL_TWEETS.json",
    "CONFIG_FEDERATED_TWEETS.json",
    testing = False
)

In [ ]:
federated.train(5)

In [ ]:
dir_ = os.path.join('tweets', 'LICCHAVI_avg')
res = {}
for i,name in enumerate(os.listdir(dir_)):
    with open(os.path.join(dir_, name, 'metrics.pickle'), 'rb') as f:
        res[i] = pickle.load(f)

In [ ]:
res.keys()

In [ ]:
f, ax = plt.subplots(1)
ax.set_ylim([0, 80])
for k,v in pd.DataFrame(res[0]).T.to_dict().items():
    if 'perplexity' == k:
        plt.plot(list(v.values()))

In [ ]:
f, ax = plt.subplots(1)
ax.set_ylim([0, 1])
for k,v in pd.DataFrame(res[0]).T.to_dict().items():
    if 'f3_recall' == k:
        plt.plot(list(v.values()))

In [ ]:
f, ax = plt.subplots(1)
ax.set_ylim([0, 1])
for k,v in pd.DataFrame(res[0]).T.to_dict().items():
    if 'f3' in k:
        plt.plot(list(v.values()))

In [ ]:
federated.nodes[7].losses